In [1]:
import re
import numpy as np
import pandas as pd
import os

In [2]:
current_dir = os.getcwd()

# 遍历当前目录下的所有文件
for filename in os.listdir(current_dir):
    # 检查文件是否以.txt为扩展名
    if filename.endswith(".txt"):
        # 输出文件名
        print(filename)

clang_core_log.txt
clang_deadcode_log.txt
clang_osx_log.txt
clang_security_log.txt
clang_unix_log.txt


In [3]:
import re

def parse_log(log_file):
    file_warnings = {}
    with open(log_file, 'r') as file:
        current_file = None
        current_warnings = 0
        current_error_lines = 0
        next_line_tool_error = False
        
        for line in file:
            # Check if the line indicates processing a new file
            if line.startswith("Processing file:"):
                if current_file:
                    file_warnings[current_file] = {
                        'warnings': current_warnings,
                        'error_lines': current_error_lines,
                    }
                current_file = line.split("Processing file:")[1].strip()
                current_warnings = 0
                current_error_lines = 0
                next_line_tool_error = False
            # Check if the line contains a warning
            elif ": warning:" in line and "<setjmp.h>" not in line:
                current_warnings += 1
                # Check if the next line contains the specified comment
                if next_line_tool_error:
                    current_error_lines += 1
                # Reset the flag for the next line
                next_line_tool_error = False
            # Check if the next line contains the specified comment
            elif "/*Tool should detect this line as error*/" in line:
                next_line_tool_error = True
        
        # Add the last file
        if current_file:
            file_warnings[current_file] = {
                'warnings': current_warnings,
                'error_lines': current_error_lines
            }
                
    return file_warnings

In [6]:
def total_error(file_warnings:dict):
    def error_of_c(file_name):
        parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
        file_path = os.path.join(parent_directory, file_name)
        with open(file_path, 'r', encoding='unicode_escape') as f:
            tool_error_count=0
            lines = f.readlines()
            for line in lines:
                if "/*Tool should detect this line as error*/" in line:
                    tool_error_count += 1
            return tool_error_count
    def count_strings_with_numbered_suffix(file_path):
        parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
        file_path = os.path.join(parent_directory, file_path)
        numbers = []
        file_name = os.path.basename(file_path).split('.')[0]
        with open(file_path, 'r', encoding='unicode_escape') as file:
            content = file.read()
            # 使用正则表达式匹配所有格式为 'file_name_数字()' 的字符串
            matches = re.findall(r'\w+_(\d+)\(\)', content)
            numbers.extend(matches)
        return max(numbers)
    
    for key, _ in file_warnings.items():
        error_count=error_of_c(key)
        total_error_count=count_strings_with_numbered_suffix(key)
        file_warnings[key]['error_in_c']=error_count
        file_warnings[key]['test_in_c']=int(total_error_count)
    return file_warnings

In [7]:
log_file = 'clang_core_log.txt'
file_warnings = parse_log(log_file)
file_warnings=total_error(file_warnings)

print("File-wise Warning Counts:")
for file_name, counts in file_warnings.items():
    print(f"File: {file_name}")
    print(f"Total Error: {counts['error_in_c']}")
    print(f"Total Warnings: {counts['warnings']}")
    print(f"Warnings with 'Tool should detect this line as error': {counts['error_lines']}")
    print(f"All tests in C': {counts['test_in_c']}")
    TP=counts['error_lines']
    FP=counts['warnings']-counts['error_lines']
    FN=counts['error_in_c']-counts['error_lines']
    TN=counts['test_in_c']-TP-FP-FN
    print(f'TP:{TP}, FP:{FP}, TN:{TN}, FN:{FN}')
    print()

File-wise Warning Counts:
File: buffer_underrun_dynamic.c
Total Error: 39
Total Warnings: 3
Warnings with 'Tool should detect this line as error': 2
All tests in C': 39
TP:2, FP:1, TN:-1, FN:37

File: cmp_funcadr.c
Total Error: 2
Total Warnings: 2
Warnings with 'Tool should detect this line as error': 1
All tests in C': 2
TP:1, FP:1, TN:-1, FN:1

File: conflicting_cond.c
Total Error: 10
Total Warnings: 0
Warnings with 'Tool should detect this line as error': 0
All tests in C': 10
TP:0, FP:0, TN:0, FN:10

File: dead_lock.c
Total Error: 5
Total Warnings: 0
Warnings with 'Tool should detect this line as error': 0
All tests in C': 5
TP:0, FP:0, TN:0, FN:5

File: double_release.c
Total Error: 5
Total Warnings: 0
Warnings with 'Tool should detect this line as error': 0
All tests in C': 6
TP:0, FP:0, TN:1, FN:5

File: improper_termination_of_block.c
Total Error: 4
Total Warnings: 1
Warnings with 'Tool should detect this line as error': 0
All tests in C': 4
TP:0, FP:1, TN:-1, FN:4

File: insig

In [14]:
int('039')

39